<a href="https://colab.research.google.com/github/ranjanchoubey/FedArtML/blob/main/examples/13_Get_started_create_FL_data_label_skew_dirichlet_InSDN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Getting started with FedArtML creating FL data (label skew) and implementing an FL model

Our ***FedArTML*** tool facilitates the generation of non-IID datasets
in a controlled way to support federated learning (FL) research for federated datasets from centralized datasets. It includes classes and functions to create federated datasets from centralized data, given the **number** of **clients** and the degree of heterogeneity (**non-IID-ness**) desired.

This guide aims to **understand** using the **SplitAsFederatedData** class to create federated datasets with the **Dirichlet method** for **Label skew**. We use the `InSDN` dataset for our tests. Moreover, we include the `Flower framework` to train an FL model using the output from the FedArtML library.

**Notes:**
1. To check the source code, you can visit the [GitHub repo](https://github.com/Sapienza-University-Rome/FedArtML)
2. To check the documentation, you visit the [dedicated link](https://fedartml.readthedocs.io/en/latest/)

# Install libraries

First, installing the FedArtML and Flower libraries from Pypi (the latest version) is necessary.

In [ ]:
#@title Install FedArtML Library { display-mode: "form" }
!pip install -q fedartml

In [ ]:
#@title Install Flower Framework { display-mode: "form" }
!pip install -q flwr[simulation]

# Import libraries

In [ ]:
#@title Import Libraries & Configure Environment { display-mode: "form" }

import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
from sklearn.preprocessing import LabelEncoder, StandardScaler

from io import BytesIO
import requests

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import datasets, layers, models

import time
import pickle

import pandas as pd

from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler

import matplotlib.pyplot as plt

import seaborn as sns
sns.set(style="darkgrid", rc = {'figure.figsize':(5,7)})

from sklearn.impute import SimpleImputer

import plotly.graph_objects as go


# Garbage Collector - use it like gc.collect()
import gc

# Custom Callback To Include in Callbacks List At Training Time
class GarbageCollectorCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()

from fedartml import InteractivePlots, SplitAsFederatedData

# Make TensorFlow logs less verbose
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import flwr as fl

from typing import List, Tuple, Dict, Optional
from flwr.common import Metrics

print("✓ All libraries imported successfully!")

# Define parameters

Then it is necessary to define some parameters to run this notebook smoothly. The most important are:



*   `local_nodes_glob`: Defines the desired number of clients (local nodes) the centralized data will be divided into. In this case, we set it as 3.
*   `Alpha`: Using the "Dirichlet-based" method, we need to define the `Alpha`, which will determine the degree of heterogeneity of the client's distribution (non-IID-ness). Notice that the smaller the value of `Alpha`, the higher the non-IID-ness. In this case, we set it as 1. Those parameters can be changed if desired to get familiar with their use.

In [ ]:
#@title Initialization { display-mode: "form" }

# Define random state for reproducibility
random_state = 0  # @param {type: "integer", min: 0, max: 10000, step: 1}

# Define colors to use in plots
colors = ["#00cfcc","#e6013b","#007f88","#00cccd","#69e0da","darkblue","#FFFFFF"]

# Path to InSDN dataset
dataset_path = '../data/LINK_all_features_all_datsets/InSDN'

print("\n✓ Initialization complete. Load data and then configure federated learning parameters.")

# Define functions

For the sake of order, we defined some functions that will be used in this guide.

In [ ]:
#@title Define Helper Functions { display-mode: "form" }

# Define function to test a model and retrieve classification metrics
def test_model(model, X_test, Y_test):
    cce = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = False)
    logits = model.predict(X_test, batch_size=32, verbose=1, callbacks=[GarbageCollectorCallback()])
    y_pred = tf.argmax(logits, axis=1)
    loss = cce(Y_test, logits).numpy()
    acc = accuracy_score(y_pred, Y_test)
    pre = precision_score(y_pred, Y_test, average='weighted',zero_division = 0)
    rec = recall_score(y_pred, Y_test, average='weighted',zero_division = 0)
    f1s = f1_score(y_pred, Y_test, average='weighted',zero_division = 0)

    return loss, acc, pre, rec, f1s

# Define function to convert from SplitAsFederatedData function output (FedArtML) to Flower (list) format
def from_FedArtML_to_Flower_format(clients_dict):
  # initialize list that contains clients (features and labels) to extract later from client_fn in Flower
  list_x_train = []
  list_y_train = []

  # Get the name of the clients from the dictionary
  client_names = list(clients_dict.keys())

  # Iterate over each client
  for client in client_names:
    # Get data from each client
    each_client_train=np.array(clients_dict[client],dtype=object)

    # Extract features for each client
    feat=[]
    x_tra=np.array(each_client_train[:, 0])
    for row in x_tra:
      feat.append(row)
    feat=np.array(feat)

    # Extract labels from each client
    y_tra=np.array(each_client_train[:, 1])

    # Append in list features and labels to extract later from client_fn in Flower
    list_x_train.append(feat)
    list_y_train.append(y_tra)

  return list_x_train, list_y_train

# Define Dense Neural Network (DNN) model for tabular InSDN data
def DNN_model(input_shape, num_classes):
  model = models.Sequential()
  model.add(layers.Input(shape=(input_shape,)))
  model.add(layers.Dense(128, activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.Dropout(0.3))
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.Dropout(0.3))
  model.add(layers.Dense(32, activation='relu'))
  model.add(layers.Dropout(0.2))
  model.add(layers.Dense(num_classes, activation='softmax'))

  return model


# Define local training/evaluation function
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, model, x_train, y_train, x_test, y_test, epochs_client) -> None:
        self.model = model
        self.x_train, self.y_train = x_train, y_train
        self.x_test, self.y_test = x_test, y_test
        self.epochs_client = epochs_client

    def get_parameters(self, config):
        return self.model.get_weights()

    def fit(self, parameters, config):
        self.model.set_weights(parameters)
        self.model.fit(self.x_train, self.y_train, validation_split=0.1, epochs=self.epochs_client, verbose=2)
        return self.model.get_weights(), len(self.x_train), {}

    def evaluate(self, parameters, config):
        self.model.set_weights(parameters)
        loss, acc = self.model.evaluate(self.x_test, self.y_test, verbose=2)
        return loss, len(self.x_test), {"accuracy": acc}

def plot_metric_from_history(
    hist: None,
    save_plot_path: None,
    metric_type: None,
    metric: None,
) -> None:
    """Function to plot from Flower server History.
    Parameters
    ----------
    hist : History
        Object containing evaluation for all rounds.
    save_plot_path : Path
        Folder to save the plot to.
    metric_type : Literal["centralized", "distributed"]
        Type of metric to plot.
    metric : Literal["accuracy","precision","recall","f1score"]
        Metric to plot.
    """
    metric_dict = (
        hist.metrics_centralized
        if metric_type == "centralized"
        else hist.metrics_distributed
    )
    rounds, values = zip(*metric_dict[metric])
    # plt.plot(np.asarray(rounds), np.asarray(values), label="FedAvg")
    plt.plot(np.asarray(rounds), np.asarray(values), color=colors[5], linewidth=5, label='Test')
    plt.legend(fontsize=45)
    plt.xlabel('Communication round', fontsize=40)
    plt.ylabel(metric, fontsize=50)
    plt.title(metric, fontsize=60)
    plt.xticks(fontsize=30)
    plt.yticks(fontsize=30)
    # plt.ylim(min(min(min(commun_metrics))) - 0.05, max(max(max(commun_metrics))) + 0.05)
    plt.ylim(0, 1)
    # plt.savefig(Path(save_plot_path) / Path(f"{metric_type}_metrics{suffix}.png"))
    # plt.close()

def retrieve_global_metrics(
    hist: None,
    metric_type: None,
    metric: None,
    best_metric: None,
) -> None:
    """Function to retrieve metrics from Flower server History.
    Parameters
    ----------
    hist : History
        Object containing evaluation for all rounds.
    metric_type : Literal["centralized", "distributed"]
        Type of metric to retrieve.
    metric : Literal["accuracy","precision","recall","f1score"]
        Metric to retrieve.
    """
    metric_dict = (
        hist.metrics_centralized
        if metric_type == "centralized"
        else hist.metrics_distributed
    )
    rounds, values = zip(*metric_dict[metric])
    if best_metric:
      metric_return = max(values)
    else:
      metric_return = values[-1]
    return metric_return

print("✓ All helper functions defined successfully!")

# Load and preprocess InSDN data

The InSDN dataset consists of network traffic data with various features. We will load, preprocess, and split the data into training and test sets.

In [ ]:
#@title Load InSDN Dataset from Official Source { display-mode: "form" }

# Load InSDN dataset from official UCD source
# InSDN: SDN Intrusion Detection Dataset
# Source: https://aseados.ucd.ie/datasets/SDN/
# Paper: "InSDN: SDN Intrusion Dataset" IEEE Access Vol.8, pp 165263-165284, Sep 2020

import zipfile
import io

print("Loading InSDN dataset...")
print("Dataset Information:")
print("  - Total instances: 343,939 network flow records")
print("  - Normal traffic: 68,424 (20%)")
print("  - Attack traffic: 275,515 (80%)")
print("  - Features: 80+ network flow characteristics")
print("  - Attack types: DoS, DDoS, Probe, Brute Force, Exploitation, Web Attack, Botnet")

df = None

# Option 1: Try to load from local path first
try:
    if os.path.exists(dataset_path):
        data_files = [f for f in os.listdir(dataset_path) if f.endswith('.csv')]
        if data_files:
            print(f"\n✓ Found local dataset: {data_files[0]}")
            df = pd.read_csv(os.path.join(dataset_path, data_files[0]))
except Exception as e:
    print(f"Local path check failed: {e}")

# Option 2: Download from official UCD source
if df is None:
    try:
        print("\n✓ Attempting to download official InSDN dataset from UCD...")
        zip_url = "https://aseados.ucd.ie/datasets/SDN/InSDN_DatasetCSV.zip"
        
        response = requests.get(zip_url, timeout=30)
        if response.status_code == 200:
            print("  Download successful! Extracting dataset...")
            
            # Extract and load the zip file
            zip_file = zipfile.ZipFile(io.BytesIO(response.content))
            csv_files = [f for f in zip_file.namelist() if f.endswith('.csv')]
            
            if csv_files:
                # Prefer OVS.csv for more attack variety
                selected_file = next((f for f in csv_files if 'OVS.csv' in f), csv_files[0])
                print(f"  Loading: {selected_file}")
                df = pd.read_csv(zip_file.open(selected_file))
                print(f"✓ Successfully loaded InSDN dataset! Shape: {df.shape}")
    except Exception as e:
        print(f"Download failed: {e}")

# If download fails, provide instructions
if df is None or df.empty:
    print("\n" + "="*80)
    print("Could not load dataset automatically")
    print("="*80)
    print("\nPlease download manually from:")
    print("  https://aseados.ucd.ie/datasets/SDN/")
    print("\nSteps:")
    print("  1. Download: InSDN_DatasetCSV.zip")
    print("  2. Extract the ZIP file")
    print("  3. Choose one CSV file:")
    print("     - OVS.csv (136,743 records - recommended for variety)")
    print("     - metasploitable-2.csv (138,772 records)")
    print("     - Normal_data.csv (68,424 records - benign traffic only)")
    print(f"  4. Place in: {os.path.abspath(dataset_path)}")
    print("="*80)
    df = pd.DataFrame()

# Display dataset information
if not df.empty:
    print("\n" + "="*80)
    print("InSDN Dataset Loaded Successfully!")
    print("="*80)
    print(f"Shape: {df.shape[0]} records x {df.shape[1]} features")
    print(f"\nFirst few rows:")
    print(df.head())
    print(f"\nColumn names ({len(df.columns)} total):")
    print(df.columns.tolist())


In [ ]:
#@title Preprocess InSDN Data { display-mode: "form" }

# Preprocess InSDN data
if not df.empty:
    print("\n" + "="*80)
    print("Preprocessing InSDN Data")
    print("="*80)
    
    print(f"\nDataFrame shape: {df.shape}")
    print(f"Column names (first 10): {list(df.columns[:10])}")
    
    # The InSDN dataset has 'Label' column (uppercase L)
    if 'Label' in df.columns:
        y_raw = df['Label'].values
        print(f"\n✓ Found 'Label' column")
        print(f"  Unique labels: {np.unique(y_raw)}")
        print(f"  Label counts: {pd.Series(y_raw).value_counts().to_dict()}")
        
        # Drop the Label column and non-numeric metadata columns
        X_df = df.drop(columns=['Label', 'Flow ID', 'Src IP', 'Dst IP', 'Timestamp'], errors='ignore')
    else:
        print("ERROR: 'Label' column not found!")
        print(f"Available columns: {list(df.columns)}")
        raise ValueError("'Label' column not found in dataset")
    
    # Select only numeric columns for features
    X_df = X_df.select_dtypes(include=[np.number])
    print(f"\nNumeric features shape: {X_df.shape}")
    print(f"Feature columns: {list(X_df.columns[:10])}... (showing first 10)")
    
    X = X_df.values
    
    # Convert labels to numeric (they are strings like 'BFA', 'DoS', etc.)
    unique_labels = sorted(set(y_raw))
    label_mapping = {label: idx for idx, label in enumerate(unique_labels)}
    y = np.array([label_mapping[label] for label in y_raw]).astype(int)
    
    print(f"\nLabel mapping ({len(label_mapping)} classes):")
    for label, idx in label_mapping.items():
        print(f"  {label} -> {idx}")
    
    print(f"\nLabel information:")
    print(f"  - Unique numeric labels: {sorted(np.unique(y))}")
    print(f"  - Label range: [{y.min()}, {y.max()}]")
    
    # Check class distribution
    unique_classes, class_counts = np.unique(y, return_counts=True)
    print(f"\nOriginal class distribution:")
    for cls, count in zip(unique_classes, class_counts):
        original_label = [k for k, v in label_mapping.items() if v == cls][0]
        print(f"  Class {cls} ({original_label}): {count} samples ({100*count/len(y):.1f}%)")
    
    # Handle missing values in features
    imputer = SimpleImputer(strategy='mean')
    X = imputer.fit_transform(X)
    print("\n✓ Missing values handled (mean imputation)")
    
    # Normalize features
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    print("✓ Features normalized (StandardScaler)")
    
    # Remove classes with very few samples (< 2) to avoid stratification issues
    min_samples_per_class = 2
    valid_classes = unique_classes[class_counts >= min_samples_per_class]
    mask = np.isin(y, valid_classes)
    X = X[mask]
    y = y[mask]
    
    print(f"\nFiltered class distribution (removed classes with < {min_samples_per_class} samples):")
    unique_classes, class_counts = np.unique(y, return_counts=True)
    for cls, count in zip(unique_classes, class_counts):
        original_label = [k for k, v in label_mapping.items() if v == cls][0]
        print(f"  Class {cls} ({original_label}): {count} samples ({100*count/len(y):.1f}%)")
    
    # Split into train and test sets
    # Use stratified split if possible, otherwise use random split
    try:
        x_train_glob, x_test_glob, y_train_glob, y_test_glob = train_test_split(
            X, y, test_size=0.2, random_state=random_state, stratify=y
        )
        print("\n✓ Used stratified train-test split")
    except ValueError:
        # If stratification fails, use random split
        x_train_glob, x_test_glob, y_train_glob, y_test_glob = train_test_split(
            X, y, test_size=0.2, random_state=random_state, stratify=None
        )
        print("✓ Used random train-test split (stratification not possible)")
    
    # Get dataset statistics
    num_classes = len(np.unique(y_train_glob))
    input_shape = x_train_glob.shape[1]
    
    print(f"\nDataset Statistics:")
    print(f"  - Training set: {x_train_glob.shape}")
    print(f"  - Test set: {x_test_glob.shape}")
    print(f"  - Number of classes: {num_classes}")
    print(f"  - Input shape: {input_shape}")
    print(f"  - Label range (train): [{y_train_glob.min()}, {y_train_glob.max()}]")
    print(f"  - Label range (test): [{y_test_glob.min()}, {y_test_glob.max()}]")
    print(f"\nClass distribution in training set:")
    unique, counts = np.unique(y_train_glob, return_counts=True)
    for cls, count in zip(unique, counts):
        pct = 100 * count / len(y_train_glob)
        original_label = [k for k, v in label_mapping.items() if v == cls][0]
        print(f"    Class {cls} ({original_label}): {count} samples ({pct:.1f}%)")
else:
    print("Cannot preprocess - dataset is empty")


In [ ]:
#@title Visualize Class Distributions { display-mode: "form" }

# Visualize class distributions
if not df.empty:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Training set
    unique_train, counts_train = np.unique(y_train_glob, return_counts=True)
    axes[0].bar(unique_train, counts_train, color=colors[0], alpha=0.7, edgecolor='black')
    axes[0].set_xlabel('Label', fontsize=12)
    axes[0].set_ylabel('Count', fontsize=12)
    axes[0].set_title('Centralized Datatset : Training Set Label Distribution', fontsize=14)
    axes[0].grid(axis='y', alpha=0.3)
    
    # Test set
    unique_test, counts_test = np.unique(y_test_glob, return_counts=True)
    axes[1].bar(unique_test, counts_test, color=colors[1], alpha=0.7, edgecolor='black')
    axes[1].set_xlabel('Label', fontsize=12)
    axes[1].set_ylabel('Count', fontsize=12)
    axes[1].set_title('Centralized Datatset : Test Set Label Distribution', fontsize=14)
    axes[1].grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()


# Create the federated dataset

This core function allows the creation of FL data from centralized data. The class `SplitAsFederatedData` is instantiated using a `random_state` to reproduce the results.

Then, the `.create_clients` function performs the federation of the centralized data by taking the features and labels, defining the number of desired clients and setting the Alpha value.

**Note:** When creating federated data and setting heterogeneous distributions (i.e. high values of percent_noniid or small values of Alpha), it is more likely the clients hold examples from only one class. Then, two cases are returned as output for fed_data and distances:
- "with_class_completion": In this case, the clients are completed with one (random) example of each missing class for each client to have all the label's classes.
- "without_class_completion": In this case, the clients are NOT completed with one (random) example of each missing class.

In [ ]:
#@title FEDERATED LEARNING CONFIGURATION: Set Parameters Here { display-mode: "form" }

# ================================================================================
# USER INPUT - SET THESE PARAMETERS ONLY ONCE
# These will be used throughout the entire notebook for federated dataset creation
# ================================================================================

# Number of clients/local nodes for federated learning
num_clients = 5  # @param {type: "integer", min: 2, max: 50, step: 1}

# Distribution method for data heterogeneity
distribution_method = "dirichlet"  # @param ["dirichlet", "percent_noniid", "no-label-skew"]

# Heterogeneity parameter (alpha)
# - For Dirichlet: controls concentration (smaller = more non-IID)
# - For Percent Non-IID: converted to percentage (0-100)
# - For IID: ignored (uniform distribution)
alpha = 1  # @param {type: "number", min: 0.001, max: 10.0, step: 0.1}

# Class completion strategy
# - "with_class_completion": Each client gets at least one sample from all classes
# - "without_class_completion": Clients may have missing classes (true non-IID scenario)
class_completion = "without_class_completion"  # @param ["with_class_completion", "without_class_completion"]

# ================================================================================
# DISPLAY CONFIGURATION SUMMARY
# ================================================================================

print("\n" + "="*80)
print("FEDERATED LEARNING CONFIGURATION - Ready to Create Dataset")
print("="*80)
print(f"\n  ✓ Number of Clients: {num_clients}")
print(f"  ✓ Distribution Method: {distribution_method.upper()}")
print(f"  ✓ Heterogeneity Parameter (alpha): {alpha}")
print(f"  ✓ Class Completion: {class_completion.upper()}")

print("\n" + "-"*80)
print("SELECTED METHOD DETAILS:")
print("-"*80)

if distribution_method == "dirichlet":
    print(f"\n  METHOD: Dirichlet Distribution (LABEL SKEW)")
    print(f"  Alpha = {alpha}")
    print(f"\n  What it does:")
    print(f"    • Each client receives a different subset of labels")
    print(f"    • Smaller alpha → More non-IID (heterogeneous)")
    print(f"    • Larger alpha → More balanced (approaching IID)")

elif distribution_method == "percent_noniid":
    percent_val = max(0, min(100, (alpha / 10.0) * 100))
    print(f"\n  METHOD: Percent Non-IID Distribution")
    print(f"  Alpha = {alpha} (converted to {percent_val:.1f}% non-IID)")
    print(f"\n  What it does:")
    print(f"    • Each client has different class distributions")
    print(f"    • Based on percentage of non-IID samples")

elif distribution_method == "no-label-skew":
    print(f"\n  METHOD: No Label Skew (IID Distribution)")
    print(f"  Alpha parameter: IGNORED")
    print(f"\n  What it does:")
    print(f"    • Each client gets balanced class distribution")
    print(f"    • All clients have similar label proportions")

print("\n" + "-"*80)
print("CLASS COMPLETION STRATEGY:")
print("-"*80)

if class_completion == "with_class_completion":
    print(f"\n  ✓ WITH Class Completion")
    print(f"  Each client will receive at least 1 sample from every class")
    print(f"  • Ensures all classes are represented in each client")
    print(f"  • Reduces data heterogeneity")
    print(f"  • Useful for testing federated learning algorithms")
else:
    print(f"\n  ✓ WITHOUT Class Completion")
    print(f"  Clients may have missing classes (true non-IID scenario)")
    print(f"  • Reflects real-world federated scenarios")
    print(f"  • Higher data heterogeneity")
    print(f"  • More challenging for federated learning")

print("\n" + "="*80)
print(f"✓ Ready to create {num_clients} clients with:")
print(f"  - Distribution: {distribution_method}")
print(f"  - Class Completion: {class_completion}")
print("="*80 + "\n")

In [ ]:
#@title Instantiate a SplitAsFederatedData object { display-mode: "form" }
if not df.empty:
    # Instantiate a SplitAsFederatedData object
    my_federater = SplitAsFederatedData(random_state = random_state)

    print("\n" + "="*80)
    print("CREATING FEDERATED DATASET")
    print("="*80)
    print(f"Method: {distribution_method}")
    print(f"Number of clients: {num_clients}")
    print(f"Class Completion: {class_completion}")
    
    # Create clients based on selected distribution method
    if distribution_method == "dirichlet":
        # Dirichlet: use alpha directly (0.001-10.0)
        print(f"Alpha: {alpha}")
        clients_glob_dic, list_ids_sampled_dic, miss_class_per_node, distances = my_federater.create_clients(
            image_list = x_train_glob, 
            label_list = y_train_glob,
            num_clients = num_clients,
            prefix_cli='Local_node',
            method = "dirichlet",
            alpha = alpha
        )
        print(f"\n✓ Dirichlet distribution created successfully")
        
    elif distribution_method == "percent_noniid":
        # Percent Non-IID: convert alpha to percentage (0-100)
        percent_value = max(0, min(100, (alpha / 10.0) * 100))
        print(f"Alpha: {alpha} → {percent_value:.1f}% non-IID")
        clients_glob_dic, list_ids_sampled_dic, miss_class_per_node, distances = my_federater.create_clients(
            image_list = x_train_glob, 
            label_list = y_train_glob,
            num_clients = num_clients,
            prefix_cli='Local_node',
            method = "percent_noniid",
            alpha = percent_value
        )
        print(f"\n✓ Percent Non-IID distribution created successfully")
        
    elif distribution_method == "no-label-skew":
        # IID: ignore alpha (uniform distribution)
        print(f"Alpha: IGNORED (IID distribution)")
        clients_glob_dic, list_ids_sampled_dic, miss_class_per_node, distances = my_federater.create_clients(
            image_list = x_train_glob, 
            label_list = y_train_glob,
            num_clients = num_clients,  # USE num_clients FROM USER INPUT
            prefix_cli='Local_node',
            method = "no-label-skew",
            alpha = None
        )
        print(f"\n✓ IID (no label skew) distribution created successfully")

    clients_glob = clients_glob_dic[class_completion]
    list_ids_sampled = list_ids_sampled_dic[class_completion]

    # Convert from SplitAsFederatedData function output (FedArtML) to Flower (list) format
    list_x_train, list_y_train = from_FedArtML_to_Flower_format(clients_dict=clients_glob)
    
    print(f"\nFederated dataset statistics:")
    print(f"  - Number of clients created: {len(list_x_train)}")
    print(f"  - Training samples per client: {[len(x) for x in list_x_train]}")
    print("="*80 + "\n")

In [ ]:
#@title Calculate Distances { display-mode: "form" }

if not df.empty:
    # Calculate distances
    JSD_glob = distances[class_completion]['jensen-shannon']
    print("Jensen-Shannon distance:", JSD_glob)
    HD_glob = distances[class_completion]['hellinger']
    print("Hellinger distance:", HD_glob)

    EMD_glob = distances[class_completion]['earth-movers']
    print("Earth Mover's distance:", EMD_glob)

In [ ]:
#@title Calculate Distances { display-mode: "form" }
if not df.empty:
    # ================================================================================
    # STANDARD COLOR PALETTE VISUALIZATION - Professional & Accessible
    # ================================================================================
    
    # Use standard professional color palettes from seaborn/matplotlib
    # These are tested, accessible, and widely used in scientific visualization
    standard_palette = sns.color_palette("husl", len(label_mapping))
    
    # Create mapping of labels to standard colors
    label_to_standard_color = {
        label: standard_palette[idx] for idx, label in enumerate(sorted(label_mapping.keys()))
    }
    
    print("\n" + "="*100)
    print("CLIENT DISTRIBUTION VISUALIZATION - STANDARD COLOR PALETTE")
    print("="*100)
    
    # Prepare data
    reverse_label_mapping = {v: k for k, v in label_mapping.items()}
    standard_plot_data = {}
    
    for key, value in clients_glob.items():
        labels_check = []
        for i in range(len(value)):
            val = value[i][1]
            labels_check.append(val)
        
        labels_check = pd.DataFrame(labels_check, columns=["label"]).reset_index()
        group = labels_check.groupby(['label']).count().reset_index().sort_values(by=['label'], ascending=True)
        group['particip'] = (group['index'].values / sum(group['index'].values)) * 100
        group.sort_values(by=['particip'], ascending=True, inplace=True)
        group['label_name'] = group['label'].map(reverse_label_mapping)
        
        # Map standard colors for each label
        group['color'] = group['label_name'].map(label_to_standard_color)
        standard_plot_data[key] = group
    
    # Get all unique labels to ensure every client shows all classes
    all_labels_sorted = sorted(label_mapping.keys())
    
    # Calculate responsive layout (max 5 clients per row)
    clients_per_row = min(5, num_clients)
    num_rows = int(np.ceil(num_clients / clients_per_row))
    
    # Calculate figure size based on number of clients
    fig_width = max(20, clients_per_row * 4.5)
    fig_height = max(14, num_rows * 5 + 3)  # +3 for legend space, increased height per row
    
    # Create figure with space for legend at bottom
    fig = plt.figure(figsize=(fig_width, fig_height))
    gs = fig.add_gridspec(num_rows + 1, clients_per_row, 
                          height_ratios=[1] * num_rows + [0.25], 
                          hspace=0.55, wspace=0.35)  # Increased hspace for more vertical separation
    
    # Plot all clients with standard palette (vertical bars)
    cont = 0
    for key, group in standard_plot_data.items():
        # Calculate row and column position
        row = cont // clients_per_row
        col = cont % clients_per_row
        ax = fig.add_subplot(gs[row, col])
        
        # Create a dataframe with ALL labels, filling missing ones with 0
        all_labels_df = pd.DataFrame({'label_name': all_labels_sorted})
        group_full = all_labels_df.merge(
            group[['label_name', 'particip', 'color']], 
            on='label_name', 
            how='left'
        )
        group_full['particip'] = group_full['particip'].fillna(0)
        # Fill missing colors
        group_full['color'] = group_full['label_name'].map(label_to_standard_color)
        
        # Create vertical bar plot with standard colors
        x_pos = np.arange(len(group_full))
        bars = ax.bar(x_pos, group_full['particip'].values, color=group_full['color'].values, 
                      edgecolor='black', linewidth=1.2, alpha=0.8)
        
        # Add percentage values on bars (only if > 0)
        for i, (x, value) in enumerate(zip(x_pos, group_full['particip'].values)):
            if value > 0:
                ax.text(x, value + 1, f'{value:.1f}%', ha='center', fontsize=8, 
                       fontweight='bold', color='#333333')
            else:
                # Show 0% for classes with no samples
                ax.text(x, 0.5, '0%', ha='center', fontsize=7, 
                       color='#999999', style='italic')
        
        # Customize axes
        ax.set_xticks(x_pos)
        ax.set_xticklabels(group_full['label_name'].values, fontsize=9, fontweight='bold', rotation=45, ha='right')
        ax.set_ylabel('Participation (%)', fontsize=10, fontweight='bold')
        ax.set_title(f'{key.replace("_", " ")}', fontsize=11, fontweight='bold')
        ax.set_ylim(bottom=0, top=max(group_full['particip']) + 15 if max(group_full['particip']) > 0 else 20)
        ax.grid(axis='y', alpha=0.3, linestyle='--')
        ax.set_axisbelow(True)
        ax.set_facecolor('#fafafa')
        
        cont += 1
    
    # Add legend at the bottom spanning all columns
    # Create a merged subplot for legend
    ax_legend = fig.add_subplot(gs[num_rows, :])
    ax_legend.axis('off')
    
    # Create legend patches
    legend_patches = []
    for label in all_labels_sorted:
        color = label_to_standard_color[label]
        legend_patches.append(plt.Rectangle((0, 0), 1, 1, facecolor=color, edgecolor='black', linewidth=1.2, alpha=0.8))
    
    # Create horizontal legend at bottom
    legend = ax_legend.legend(
        legend_patches, 
        all_labels_sorted,
        loc='center',
        ncol=len(all_labels_sorted),
        fontsize=10,
        title='Class Labels (Color Code)',
        title_fontsize=11,
        frameon=True,
        fancybox=True,
        shadow=True
    )
    
    # Overall figure styling
    fig.suptitle('Client Label Distribution - Standard Color Palette (Vertical Chart with Legend)', 
                fontsize=16, fontweight='bold', y=0.99)
    
    plt.show()
    
    # ================================================================================
    # DISPLAY STANDARD PALETTE INFORMATION
    # ================================================================================
    print("\n[STANDARD PALETTE] Using 'husl' colormap from seaborn")
    print("-" * 100)
    print(f"✓ Number of classes: {len(label_mapping)}")
    print(f"✓ Palette: HLS (Hue, Lightness, Saturation) - Professional & Accessible")
    print(f"\nColor assignments:")
    for label in sorted(label_mapping.keys()):
        color_rgb = label_to_standard_color[label]
        color_hex = '#{:02x}{:02x}{:02x}'.format(
            int(color_rgb[0]*255), int(color_rgb[1]*255), int(color_rgb[2]*255)
        )
        print(f"  • {label:<15} → {color_hex}")
    
    print("\n" + "="*100)
    print("✓ Visualization complete using standard professional color palette")
    print("="*100 + "\n")


In [ ]:
#@title Export Client Data as Tables for Report { display-mode: "form" }

if not df.empty:
    # ================================================================================
    # EXPORT CLIENT DISTRIBUTION DATA AS PROFESSIONAL TABLE FOR REPORT
    # ================================================================================
    
    print("\n" + "="*100)
    print("CLIENT DISTRIBUTION DATA - PROFESSIONAL TABLE FOR REPORT")
    print("="*100 + "\n")
    
    # Create comprehensive report data with samples and percentages
    report_data = []
    
    for client_idx, (key, group) in enumerate(standard_plot_data.items(), 1):
        # Get all labels for this client
        all_labels_df = pd.DataFrame({'label_name': all_labels_sorted})
        group_full = all_labels_df.merge(
            group[['label_name', 'particip']], 
            on='label_name', 
            how='left'
        )
        group_full['particip'] = group_full['particip'].fillna(0)
        
        # Get total samples for this client
        client_data = clients_glob[key]
        total_samples = len(client_data)
        
        # Create row for each class
        for label_name, participation in zip(group_full['label_name'], group_full['particip']):
            num_samples = int((participation / 100) * total_samples) if participation > 0 else 0
            report_data.append({
                'Client': key.replace('_', ' '),
                'Class': label_name,
                'Participation (%)': f"{participation:.2f}",
                'Samples': num_samples
            })
    
    # Create DataFrame
    report_df = pd.DataFrame(report_data)
    
    # Create a pivot table for better visualization (Clients as rows, Classes as columns)
    pivot_table = report_df.pivot_table(
        index='Client',
        columns='Class',
        values='Participation (%)',
        aggfunc='first'
    ).fillna('0.00')
    
    # Create a pivot table for samples as well
    pivot_samples = report_df.pivot_table(
        index='Client',
        columns='Class',
        values='Samples',
        aggfunc='first'
    ).fillna(0).astype(int)
    
    # Convert to numeric for display
    pivot_table_numeric = pivot_table.astype(float)
    
    # Create a professional matplotlib table visualization with samples and percentages
    fig, ax = plt.subplots(figsize=(18, 7))
    ax.axis('tight')
    ax.axis('off')
    
    # Prepare table data with both percentages and sample counts
    table_data = []
    table_data.append(['Client'] + list(pivot_table.columns))
    
    for client_idx, row in pivot_table.iterrows():
        row_data = [client_idx]
        for col_idx, col in enumerate(pivot_table.columns):
            percentage = float(row[col])
            samples = int(pivot_samples.loc[client_idx, col])
            # Format as "samples (percentage%)"
            cell_value = f"{samples}\n({percentage:.1f}%)"
            row_data.append(cell_value)
        table_data.append(row_data)
    
    # Create table
    table = ax.table(
        cellText=table_data,
        cellLoc='center',
        loc='center',
        colWidths=[0.15] + [0.12] * len(pivot_table.columns)
    )
    
    # Style the table
    table.auto_set_font_size(False)
    table.set_fontsize(9)
    table.scale(1, 3)
    
    # Color header row
    for i in range(len(pivot_table.columns) + 1):
        cell = table[(0, i)]
        cell.set_facecolor('#2C5AA0')
        cell.set_text_props(weight='bold', color='white', fontsize=11)
    
    # Color client name column
    for i in range(1, len(pivot_table) + 1):
        cell = table[(i, 0)]
        cell.set_facecolor('#E8F0F7')
        cell.set_text_props(weight='bold', fontsize=10)
    
    # Alternate row colors for data cells
    for i in range(1, len(pivot_table) + 1):
        for j in range(1, len(pivot_table.columns) + 1):
            cell = table[(i, j)]
            if i % 2 == 0:
                cell.set_facecolor('#F5F5F5')
            else:
                cell.set_facecolor('#FFFFFF')
            
            # Highlight high participation values
            try:
                cell_text = table_data[i][j]
                # Extract percentage from "samples (percentage%)"
                percentage_str = cell_text.split('(')[1].replace('%)', '')
                val = float(percentage_str)
                if val > 30:
                    cell.set_text_props(weight='bold', color='#D32F2F')
                elif val > 0:
                    cell.set_text_props(color='#1976D2')
            except:
                pass
    
    plt.title('Client Label Distribution - Samples (Participation %)', 
              fontsize=14, fontweight='bold', pad=20)
    
    plt.tight_layout()
    plt.savefig('client_distribution_table.png', dpi=300, bbox_inches='tight', facecolor='white')
    plt.show()
    
    print("✓ Professional table saved as 'client_distribution_table.png'")
    print("  Format: Samples (Participation %)")
    print("  (Ready to insert into your report)\n")
    
    # Also create a summary statistics table with sample counts
    print("\n" + "-" * 100)
    print("SUMMARY STATISTICS BY CLASS")
    print("-" * 100 + "\n")
    
    summary_by_class = report_df.copy()
    summary_by_class['Participation (%)'] = summary_by_class['Participation (%)'].astype(float)
    summary_stats = summary_by_class.groupby('Class').agg({
        'Participation (%)': ['min', 'max', 'mean'],
        'Samples': 'sum'
    }).round(2)
    summary_stats.columns = ['Min (%)', 'Max (%)', 'Mean (%)', 'Total Samples']
    
    # Create summary table visualization
    fig, ax = plt.subplots(figsize=(14, len(summary_stats) * 0.7 + 1))
    ax.axis('tight')
    ax.axis('off')
    
    # Prepare summary table data
    summary_table_data = [['Class', 'Min (%)', 'Max (%)', 'Mean (%)', 'Total Samples']]
    for class_name, row in summary_stats.iterrows():
        summary_table_data.append([
            class_name,
            f"{row['Min (%)']:.2f}",
            f"{row['Max (%)']:.2f}",
            f"{row['Mean (%)']:.2f}",
            f"{int(row['Total Samples'])}"
        ])
    
    # Create summary table
    summary_table = ax.table(
        cellText=summary_table_data,
        cellLoc='center',
        loc='center',
        colWidths=[0.25, 0.15, 0.15, 0.15, 0.20]
    )
    
    # Style summary table
    summary_table.auto_set_font_size(False)
    summary_table.set_fontsize(10)
    summary_table.scale(1, 2.5)
    
    # Color header
    for i in range(5):
        cell = summary_table[(0, i)]
        cell.set_facecolor('#2C5AA0')
        cell.set_text_props(weight='bold', color='white', fontsize=11)
    
    # Color rows
    for i in range(1, len(summary_stats) + 1):
        for j in range(5):
            cell = summary_table[(i, j)]
            if i % 2 == 0:
                cell.set_facecolor('#F5F5F5')
            else:
                cell.set_facecolor('#FFFFFF')
            
            if j == 0:  # Class column
                cell.set_text_props(weight='bold')
    
    plt.title('Summary Statistics - Class Distribution Across All Clients', 
              fontsize=14, fontweight='bold', pad=20)
    
    plt.tight_layout()
    plt.savefig('class_summary_statistics_table.png', dpi=300, bbox_inches='tight', facecolor='white')
    plt.show()
    
    print("\n✓ Summary statistics table saved as 'class_summary_statistics_table.png'")
    print("  Shows Min/Max/Mean participation % and total samples per class")
    print("  (Ready to insert into your report)\n")
    
    # Export as CSV for data reference
    print("-" * 100)
    print("DATA EXPORT")
    print("-" * 100)
    print(f"✓ Full report data available in 'report_df' variable")
    print(f"✓ Pivot table (percentages) available in 'pivot_table' variable")
    print(f"✓ Pivot table (samples) available in 'pivot_samples' variable")
    print(f"✓ Export full data: report_df.to_csv('client_distribution_report.csv', index=False)")
    print(f"✓ Export summary: summary_stats.to_csv('class_summary_statistics.csv')")
    
    print("\n" + "="*100)
    print("✓ Professional tables created with sample counts and percentages")

    print("="*100 + "\n")

# Train with Dense Neural Network (DNN)

Up to this point, we have shown how to use the **FedArtML** to create a federated dataset starting from centralized data. Then, we introduce the code to train an FL model using the **Flower framework**.

In [ ]:
#@title Configure Training Parameters { display-mode: "form" }

# Configure training parameters
if not df.empty:
    # Define loss and metrics
    loss_inic = SparseCategoricalCrossentropy()
    metrics = SparseCategoricalAccuracy()
    
    # Training configuration
    epochs = 2
    comms_round = 2
    
    print("\nTraining Configuration:")
    print(f"  - Local epochs per round: {epochs}")
    print(f"  - Communication rounds: {comms_round}")
    print(f"  - Number of clients: {num_clients}")

    print(f"  - Input shape: {input_shape}")
    print(f"  - Output classes: {num_classes}")

In [ ]:
#@title Define Flower Client Function { display-mode: "form" }

if not df.empty:
    # The `evaluate` function will be by Flower called after every round
    def evaluate_DNN(
        server_round: int,
        parameters: fl.common.NDArrays,
        config: Dict[str, fl.common.Scalar],
    ) -> Optional[Tuple[float, Dict[str, fl.common.Scalar]]]:
        net = DNN_model(input_shape, num_classes)
        net.set_weights(parameters) # Update model with the latest parameters
        loss, accuracy, precision, recall, f1score  = test_model(net, x_test_glob, np.array(y_test_glob))

        print(f"@@@@@@ Server-side evaluation loss {loss} / accuracy {accuracy} / f1score {f1score} @@@@@@")
        return loss, {"accuracy": accuracy, "precision": precision, "recall": recall, "f1score": f1score}

In [ ]:
#@title Run Federated Learning Training { display-mode: "form" }

if not df.empty:
    print("\n" + "="*80)
    print("Starting Federated Learning Training")
    print("="*80)
    
    # Define client function
    def client_fn(cid: str) -> fl.client.Client:
        # Create model
        model = DNN_model(input_shape, num_classes)
        model.compile(optimizer=Adam(learning_rate=0.001), loss=loss_inic, metrics=[metrics])
        
        # Load client data partition
        x_train_cid = np.array(list_x_train[int(cid)], dtype=float)
        y_train_cid = np.array(list_y_train[int(cid)], dtype=float)
        
        return FlowerClient(model, x_train_cid, y_train_cid, x_test_glob, y_test_glob, epochs)
    
    # Create FedAvg strategy
    strategy = fl.server.strategy.FedAvg(
        fraction_fit=1.0,
        fraction_evaluate=0.5,
        min_fit_clients=num_clients,
        min_evaluate_clients=num_clients // 2,
        min_available_clients=num_clients,
        evaluate_fn=evaluate_DNN
    )
    
    # Run federated learning simulation
    start_time = time.time()
    commun_metrics_history = fl.simulation.start_simulation(
        client_fn=client_fn,
        num_clients=num_clients,
        config=fl.server.ServerConfig(num_rounds=comms_round),
        strategy=strategy,
    )
    training_time = time.time() - start_time
    
    # Extract final metrics
    global_acc_test = retrieve_global_metrics(commun_metrics_history, "centralized", "accuracy", False)
    global_pre_test = retrieve_global_metrics(commun_metrics_history, "centralized", "precision", False)
    global_rec_test = retrieve_global_metrics(commun_metrics_history, "centralized", "recall", False)
    global_f1s_test = retrieve_global_metrics(commun_metrics_history, "centralized", "f1score", False)
    
    print("\n" + "="*80)
    print("FINAL RESULTS")
    print("="*80)
    print(f"Accuracy:  {global_acc_test:.4f}")
    print(f"Precision: {global_pre_test:.4f}")
    print(f"Recall:    {global_rec_test:.4f}")
    print(f"F1-Score:  {global_f1s_test:.4f}")
    print(f"Training time: {training_time:.2f} seconds")

    print("="*80)

# Plot Results

In [ ]:
#@title Individual Client Learning Analysis { display-mode: "form" }

if not df.empty:
    # ================================================================================
    # INDIVIDUAL CLIENT LEARNING ANALYSIS
    # ================================================================================
    
    print("\n" + "="*100)
    print("INDIVIDUAL CLIENT LEARNING ANALYSIS - INSPECTING CLIENT PERFORMANCE")
    print("="*100 + "\n")
    
    # Evaluate model on each client's test set to see individual learning
    client_metrics = []
    
    for client_id in range(num_clients):
        # Get client's training data
        x_train_client = np.array(list_x_train[client_id], dtype=float)
        y_train_client = np.array(list_y_train[client_id], dtype=float)
        
        # Create and evaluate model for this client
        client_model = DNN_model(input_shape, num_classes)
        client_model.compile(optimizer=Adam(learning_rate=0.001), loss=loss_inic, metrics=[metrics])
        
        # Train on client's data
        print(f"Training Client {client_id}: {len(x_train_client)} samples...")
        client_model.fit(
            x_train_client, y_train_client,
            validation_split=0.1,
            epochs=epochs,
            verbose=0,
            callbacks=[GarbageCollectorCallback()]
        )
        
        # Evaluate on global test set
        cce = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
        logits = client_model.predict(x_test_glob, batch_size=32, verbose=0, callbacks=[GarbageCollectorCallback()])
        y_pred = tf.argmax(logits, axis=1)
        
        loss = cce(y_test_glob, logits).numpy()
        acc = accuracy_score(y_pred, y_test_glob)
        pre = precision_score(y_pred, y_test_glob, average='weighted', zero_division=0)
        rec = recall_score(y_pred, y_test_glob, average='weighted', zero_division=0)
        f1s = f1_score(y_pred, y_test_glob, average='weighted', zero_division=0)
        
        client_metrics.append({
            'Client': f'Client {client_id}',
            'Samples': len(x_train_client),
            'Loss': loss,
            'Accuracy': acc,
            'Precision': pre,
            'Recall': rec,
            'F1-Score': f1s
        })
        
        print(f"  ✓ Accuracy: {acc:.4f}, F1-Score: {f1s:.4f}\n")
    
    # Create DataFrame
    client_df = pd.DataFrame(client_metrics)
    
    print("\n" + "-"*100)
    print("INDIVIDUAL CLIENT PERFORMANCE SUMMARY")
    print("-"*100 + "\n")
    print(client_df.to_string(index=False))
    
    # Calculate statistics
    print("\n" + "-"*100)
    print("CLIENT PERFORMANCE STATISTICS")
    print("-"*100 + "\n")
    
    stats_dict = {
        'Metric': ['Loss', 'Accuracy', 'Precision', 'Recall', 'F1-Score'],
        'Min': [
            client_df['Loss'].min(),
            client_df['Accuracy'].min(),
            client_df['Precision'].min(),
            client_df['Recall'].min(),
            client_df['F1-Score'].min()
        ],
        'Max': [
            client_df['Loss'].max(),
            client_df['Accuracy'].max(),
            client_df['Precision'].max(),
            client_df['Recall'].max(),
            client_df['F1-Score'].max()
        ],
        'Mean': [
            client_df['Loss'].mean(),
            client_df['Accuracy'].mean(),
            client_df['Precision'].mean(),
            client_df['Recall'].mean(),
            client_df['F1-Score'].mean()
        ],
        'Std Dev': [
            client_df['Loss'].std(),
            client_df['Accuracy'].std(),
            client_df['Precision'].std(),
            client_df['Recall'].std(),
            client_df['F1-Score'].std()
        ]
    }
    
    stats_df = pd.DataFrame(stats_dict)
    print(stats_df.to_string(index=False))
    
    # Create professional visualization - Client Performance Comparison
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    fig.suptitle('Individual Client Learning Performance - Per-Client Evaluation', 
                 fontsize=16, fontweight='bold')
    
    metrics_to_plot = ['Accuracy', 'F1-Score', 'Precision', 'Recall', 'Loss']
    colors_palette = sns.color_palette("husl", num_clients)
    
    # Accuracy
    ax = axes[0, 0]
    bars = ax.bar(range(num_clients), client_df['Accuracy'].values, color=colors_palette, edgecolor='black', linewidth=1.2, alpha=0.8)
    ax.set_ylabel('Accuracy', fontsize=11, fontweight='bold')
    ax.set_xlabel('Client', fontsize=11, fontweight='bold')
    ax.set_title('Accuracy per Client', fontsize=12, fontweight='bold')
    ax.set_xticks(range(num_clients))
    ax.set_xticklabels([f'C{i}' for i in range(num_clients)])
    ax.grid(axis='y', alpha=0.3)
    ax.axhline(y=client_df['Accuracy'].mean(), color='red', linestyle='--', linewidth=2, label='Mean')
    ax.legend()
    for i, v in enumerate(client_df['Accuracy'].values):
        ax.text(i, v + 0.02, f'{v:.3f}', ha='center', fontsize=9, fontweight='bold')
    
    # F1-Score
    ax = axes[0, 1]
    bars = ax.bar(range(num_clients), client_df['F1-Score'].values, color=colors_palette, edgecolor='black', linewidth=1.2, alpha=0.8)
    ax.set_ylabel('F1-Score', fontsize=11, fontweight='bold')
    ax.set_xlabel('Client', fontsize=11, fontweight='bold')
    ax.set_title('F1-Score per Client', fontsize=12, fontweight='bold')
    ax.set_xticks(range(num_clients))
    ax.set_xticklabels([f'C{i}' for i in range(num_clients)])
    ax.grid(axis='y', alpha=0.3)
    ax.axhline(y=client_df['F1-Score'].mean(), color='red', linestyle='--', linewidth=2, label='Mean')
    ax.legend()
    for i, v in enumerate(client_df['F1-Score'].values):
        ax.text(i, v + 0.02, f'{v:.3f}', ha='center', fontsize=9, fontweight='bold')
    
    # Precision
    ax = axes[0, 2]
    bars = ax.bar(range(num_clients), client_df['Precision'].values, color=colors_palette, edgecolor='black', linewidth=1.2, alpha=0.8)
    ax.set_ylabel('Precision', fontsize=11, fontweight='bold')
    ax.set_xlabel('Client', fontsize=11, fontweight='bold')
    ax.set_title('Precision per Client', fontsize=12, fontweight='bold')
    ax.set_xticks(range(num_clients))
    ax.set_xticklabels([f'C{i}' for i in range(num_clients)])
    ax.grid(axis='y', alpha=0.3)
    ax.axhline(y=client_df['Precision'].mean(), color='red', linestyle='--', linewidth=2, label='Mean')
    ax.legend()
    for i, v in enumerate(client_df['Precision'].values):
        ax.text(i, v + 0.02, f'{v:.3f}', ha='center', fontsize=9, fontweight='bold')
    
    # Recall
    ax = axes[1, 0]
    bars = ax.bar(range(num_clients), client_df['Recall'].values, color=colors_palette, edgecolor='black', linewidth=1.2, alpha=0.8)
    ax.set_ylabel('Recall', fontsize=11, fontweight='bold')
    ax.set_xlabel('Client', fontsize=11, fontweight='bold')
    ax.set_title('Recall per Client', fontsize=12, fontweight='bold')
    ax.set_xticks(range(num_clients))
    ax.set_xticklabels([f'C{i}' for i in range(num_clients)])
    ax.grid(axis='y', alpha=0.3)
    ax.axhline(y=client_df['Recall'].mean(), color='red', linestyle='--', linewidth=2, label='Mean')
    ax.legend()
    for i, v in enumerate(client_df['Recall'].values):
        ax.text(i, v + 0.02, f'{v:.3f}', ha='center', fontsize=9, fontweight='bold')
    
    # Loss
    ax = axes[1, 1]
    bars = ax.bar(range(num_clients), client_df['Loss'].values, color=colors_palette, edgecolor='black', linewidth=1.2, alpha=0.8)
    ax.set_ylabel('Loss', fontsize=11, fontweight='bold')
    ax.set_xlabel('Client', fontsize=11, fontweight='bold')
    ax.set_title('Loss per Client (Lower is Better)', fontsize=12, fontweight='bold')
    ax.set_xticks(range(num_clients))
    ax.set_xticklabels([f'C{i}' for i in range(num_clients)])
    ax.grid(axis='y', alpha=0.3)
    ax.axhline(y=client_df['Loss'].mean(), color='red', linestyle='--', linewidth=2, label='Mean')
    ax.legend()
    for i, v in enumerate(client_df['Loss'].values):
        ax.text(i, v + 0.01, f'{v:.3f}', ha='center', fontsize=9, fontweight='bold')
    
    # Samples per Client
    ax = axes[1, 2]
    bars = ax.bar(range(num_clients), client_df['Samples'].values, color=colors_palette, edgecolor='black', linewidth=1.2, alpha=0.8)
    ax.set_ylabel('Number of Samples', fontsize=11, fontweight='bold')
    ax.set_xlabel('Client', fontsize=11, fontweight='bold')
    ax.set_title('Training Samples per Client', fontsize=12, fontweight='bold')
    ax.set_xticks(range(num_clients))
    ax.set_xticklabels([f'C{i}' for i in range(num_clients)])
    ax.grid(axis='y', alpha=0.3)
    for i, v in enumerate(client_df['Samples'].values):
        ax.text(i, v + 5, f'{int(v)}', ha='center', fontsize=9, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    print("\n✓ Professional visualization saved as 'individual_client_performance.png'")
    print("  (Ready to insert into your report)\n")
    

    # Identify learning patterns
    print("-"*100)
    print("LEARNING PATTERN ANALYSIS")
    print("-"*100 + "\n")
    
    # Find best and worst performing clients
    best_client = client_df.loc[client_df['Accuracy'].idxmax()]
    worst_client = client_df.loc[client_df['Accuracy'].idxmin()]
    
    print(f"✓ Best Performing Client: {best_client['Client']}")
    print(f"  Accuracy: {best_client['Accuracy']:.4f}, F1-Score: {best_client['F1-Score']:.4f}, Samples: {int(best_client['Samples'])}")
    
    print(f"\n✓ Lowest Performing Client: {worst_client['Client']}")
    print(f"  Accuracy: {worst_client['Accuracy']:.4f}, F1-Score: {worst_client['F1-Score']:.4f}, Samples: {int(worst_client['Samples'])}")
    

    

In [ ]:
#@title Performance Metrics Trend Plot { display-mode: "form" }

if not df.empty:
    # Define metrics to plot
    metrics_show = ["accuracy","precision","recall","f1score"]
    metric_labels = ["Accuracy", "Precision", "Recall", "F1-Score"]

    # Create a larger figure with better spacing
    fig = plt.figure(figsize=(24, 7))
    fig.suptitle('Federated Learning Performance Metrics', fontsize=28, fontweight='bold', y=1.00)

    # Loop over the communication round history and metrics
    for i in range(len(metrics_show)):
        ax = plt.subplot(1, len(metrics_show), i + 1)
        
        # Get metric data
        metric_dict = commun_metrics_history.metrics_centralized
        rounds, values = zip(*metric_dict[metrics_show[i]])
        
        # Plot with enhanced styling
        ax.plot(np.asarray(rounds), np.asarray(values), color=colors[5], linewidth=4, marker='o', markersize=10, label='Test')
        ax.fill_between(np.asarray(rounds), np.asarray(values), alpha=0.2, color=colors[5])
        
        # Styling
        ax.set_xlabel('Communication Round', fontsize=16, fontweight='bold')
        ax.set_ylabel(metric_labels[i], fontsize=16, fontweight='bold')
        ax.set_title(metric_labels[i], fontsize=18, fontweight='bold', pad=15)
        ax.set_ylim(-0.05, 1.08)
        ax.grid(True, alpha=0.3, linestyle='--')
        ax.legend(fontsize=14, loc='lower right')
        ax.tick_params(labelsize=13)
        ax.set_axisbelow(True)
    

    plt.tight_layout()
    plt.show()

In [ ]:
#@title Final Model Performance Metrics Bar Chart { display-mode: "form" }

if not df.empty:
    # Define metrics in a dataframe
    metrics_DNN = {'metric_name':  ['Accuracy', 'Precision', 'Recall', 'F1-Score'],
               'set_used': ['Test', 'Test', 'Test', 'Test'],
               'metric_value': [ global_acc_test , global_pre_test , global_rec_test , global_f1s_test]
            }

    metrics_DNN = pd.DataFrame(metrics_DNN, columns = ['metric_name', 'set_used','metric_value'])

    # Plot metrics with improved styling
    fig = go.Figure(data=[
        go.Bar(
            name='Test',
            x=metrics_DNN[metrics_DNN['set_used']=="Test"]['metric_name'],
            y=metrics_DNN[metrics_DNN['set_used']=="Test"]['metric_value'],
            marker_color='#0091ad',
            marker_line_color='#005f7a',
            marker_line_width=2,
            text=[f"{v:.4f}" for v in metrics_DNN[metrics_DNN['set_used']=="Test"]['metric_value']],
            textposition='outside',
            textfont=dict(size=14, color='#005f7a')
        )
    ])
    
    # Update layout for better appearance
    fig.update_layout(
        title=dict(
            text='<b>Final Federated Learning Model Performance</b>',
            font=dict(size=22, color='#005f7a'),
            x=0.5,
            xanchor='center'
        ),
        xaxis_title='<b>Metric</b>',
        yaxis_title='<b>Metric Value</b>',
        xaxis_title_font=dict(size=16),
        yaxis_title_font=dict(size=16),
        xaxis_tickfont=dict(size=14),
        yaxis_tickfont=dict(size=14),
        barmode='group',
        autosize=False,
        width=900,
        height=600,
        showlegend=False,
        plot_bgcolor='rgba(240,240,240,0.5)',
        margin=dict(l=100, r=100, t=100, b=100),
        yaxis=dict(range=[0, 1.1])

    )    fig.show()